# Benchmark Analysis

### DOI histogram ground truth

In [8]:
import pandas as pd
import altair as alt
from config import *

ground_truth_df = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df)
ground_truth_bins.columns = ["count"]
ground_truth_bins["bin"] = ground_truth_bins.index / no_bins

alt.Chart(ground_truth_bins).mark_bar().encode(
  x=alt.X("bin:N"),
  y=alt.Y("count:Q"),
).properties(
  width=400,
  height=100
)

alt.Chart(...)

### Bigger chunks vs. ground truth

In [9]:
import altair as alt
from config import *


ground_truth_df = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df, with_labels=True)

bigger_chunks_df = pd.read_csv(f"{path}/doi/__bigger_chunks__.csv")
bigger_chunks_bins = get_doi_bins_df(bigger_chunks_df, with_labels=True)

delta_bins_df = get_doi_delta_bins_df(bigger_chunks_bins[0], ground_truth_bins[0])

alt.Chart(delta_bins_df).mark_bar().encode(
  x=alt.X("bin:Q"),
  y=alt.Y("delta:Q"),
  tooltip=alt.Tooltip(["bin", "delta"]),
).properties(
  width=100,
  height=100
)


alt.Chart(...)

### DOI distributions per use case in histograms

In [10]:
import altair as alt
from config import *

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{path}/doi")

# compute the ground truth bins 
ground_truth_df = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
ground_truth_bins = get_doi_bins_df(ground_truth_df)

all_doi_bins_df = pd.DataFrame()

# compute the bins for each combination of strategies and then compare it to the ground truth in a 
# layered histogram
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # bin the doi data 
    df = pd.read_csv(f"{path}/doi/{test_case}")
    bins_df = get_doi_bins_df(df)

    doi_delta_bins = get_doi_delta_bins_df(bins_df, ground_truth_bins)

    doi_delta_bins["context_strategy"] = c_strat[0]
    doi_delta_bins["update_strategy"] = u_strat[0]

    all_doi_bins_df = all_doi_bins_df.append(doi_delta_bins)

alt.data_transformers.enable("data_server")
alt.Chart(all_doi_bins_df).mark_bar().encode(
  x=alt.X("bin:Q"),
  y=alt.Y("delta:Q"),
  tooltip=alt.Tooltip(["bin", "delta"]),
).properties(
  width=100,
  height=100
).facet(
  row="context_strategy",
  column="update_strategy",
  spacing=10
)

alt.FacetChart(...)

### DOI accuracy per item

In [8]:
from config import *
import pandas as pd
import altair as alt
from database import ID, DOI


id = ID.lower()
doi = DOI.lower()
gt = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
gt_bin_labels = get_doi_bins_df(gt, with_labels=True)[1]

available_test_cases = os.listdir(f"{path}/doi")
results = pd.DataFrame()

# compute the overlap between the two 
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df = pd.read_csv(f"{path}/doi/{test_case}")
    bin_labels = get_doi_bins_df(df, True)[1]

    diff = get_doi_bin_error_df(gt_bin_labels, bin_labels)
    diff["context_strategy"] = c_strat[0]
    diff["update_strategy"] = u_strat[0]    
    
    results = results.append(diff)

strategies_chart = alt.Chart(results).mark_bar().encode(
  x="count:Q",
  y=alt.Y("update_strategy:N", title=None),
  row="context_strategy:N",
  stroke={"value": "#fff"},
  color=alt.Color("diff:Q", scale=alt.Scale(scheme='viridis')),
  tooltip=["diff", "count"],
)

bigger_chunks_df = pd.read_csv(f"{path}/doi/__bigger_chunks__.csv")
bigger_chunks_bins = get_doi_bins_df(bigger_chunks_df, with_labels=True)

doi_bin_error_df = get_doi_bin_error_df(bigger_chunks_bins[1], gt_bin_labels)

bigger_chunks_chart = alt.Chart(doi_bin_error_df).mark_bar().encode(
  x="count:Q",
  stroke={"value": "#fff"},
  color=alt.Color("diff:Q", scale=alt.Scale(scheme='viridis')),
  tooltip=["diff", "count"],
).properties(
  title="Bigger chunks:"
)

print("doi accuracy per item:")
alt.vconcat(strategies_chart, bigger_chunks_chart)

doi accuracy per item:


alt.VConcatChart(...)

### Median error per item

In [1]:
from config import *
import pandas as pd
import altair as alt


gt = pd.read_csv(f"{path}/doi/__ground_truth__.csv")
gt_bin_labels = get_doi_bins_df(gt, with_labels=True)[1]

available_test_cases = os.listdir(f"{path}/doi")
results = pd.DataFrame()

# compute the overlap between the two 
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    # read the benchmark results
    df = pd.read_csv(f"{path}/doi/{test_case}")
    bin_labels = get_doi_bins_df(df, True)[1]

    diff = get_doi_bin_error_df(gt_bin_labels, bin_labels)
    diff["context_strategy"] = c_strat[0]
    diff["update_strategy"] = u_strat[0]    
    
    results = results.append(diff)

results["multiplied_error"] = results["diff"] * results["count"]
grouped = results.groupby(["context_strategy", "update_strategy"]).median()
grouped.reset_index(inplace=True)

strategy_chart = alt.Chart(grouped).mark_bar().encode(
  x=alt.X("multiplied_error:Q", title=None),
  y=alt.Y("update_strategy:N", title=None),
  row="context_strategy:N"
)

bigger_chunks_df = pd.read_csv(f"{path}/doi/__bigger_chunks__.csv")
bigger_chunks_bins = get_doi_bins_df(bigger_chunks_df, with_labels=True)
doi_bin_error_df = get_doi_bin_error_df(bigger_chunks_bins[1], gt_bin_labels)

doi_bin_error_df["multiplied_error"] = doi_bin_error_df["diff"] * doi_bin_error_df["count"]
grouped2 = pd.DataFrame(doi_bin_error_df.median()).T

bigger_chunks_chart = alt.Chart(grouped2).mark_bar().encode(
  x=alt.X("multiplied_error:Q", scale=alt.Scale(domain=[0, grouped["multiplied_error"].max()]))
).properties(
  title="using bigger chunks:"
)

print("median error per item")
alt.vconcat(strategy_chart, bigger_chunks_chart)

median error per item


alt.VConcatChart(...)

In [3]:
from config import *
from database import get_next_chunk_from_db
from sklearn.preprocessing import sample_without_replacement

reset()
df = get_next_chunk_from_db(1000, as_df=True)
df.columns = ["tripID", "a", "b"]

# sample = sample_without_replacement()

alt.Chart(df).mark_point().encode(
  x="a:Q",
  y="b:Q"
)

rows = []
for c_strat in context_strategies:
  columns = []
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue
    testcase_df = pd.read_csv(f"{path}/doi/{test_case}")

    # print(testcase_df)
    df["doi"] = testcase_df["doi"]
    chart = alt.Chart(df).mark_point().encode(
      x="a:Q",
      y="b:Q",
      color=alt.Color("doi:Q", scale=alt.Scale(scheme='inferno')),
      opacity={"value": 0.3},
    ).properties(
      title=f"{c_strat[0]}-{u_strat[0]}",
      width=200,
      height=200,
    )
    columns += [chart]
  rows += [alt.hconcat(*columns)]

alt.vconcat(*rows)

ImportError: cannot import name 'sample_without_replacement' from 'sklearn.preprocessing' (C:\Python39\lib\site-packages\sklearn\preprocessing\__init__.py)

### Time per test case in boxplots

In [5]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{path}/times")
available_test_cases

all_doi_values_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{path}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_doi_values_df = all_doi_values_df.append(df)
    all_doi_values_df.reset_index(inplace=True, drop=True)

chart1 = alt.Chart(all_doi_values_df).mark_boxplot().encode(
  x="update_strategy:N",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  column="context_strategy:N",
).properties(
  width=190,
  height=250
)

chart2 = alt.Chart(all_doi_values_df).mark_boxplot().encode(
  x="context_strategy:N",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  column="update_strategy:N",
).properties(
  width=190,
  height=250
)

alt.vconcat(chart1, chart2)

alt.VConcatChart(...)

### Time series per test case

In [6]:
import altair as alt

charts = []

# load all data from the out directory into one dataframe and add a column that indicates the context
# and update strategies used in this particular use case
available_test_cases = os.listdir(f"{path}/times")
available_test_cases

all_timeseries_df = pd.DataFrame()

# build one big dataframe containing all doi scores and label each based on the strategies that were
# used to generate them
for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{path}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_timeseries_df = all_timeseries_df.append(df)
    all_timeseries_df.reset_index(inplace=True, drop=True)

alt.data_transformers.disable_max_rows()
alt.Chart(all_timeseries_df).mark_line().encode(
  x="chunk:Q",
  y={"field": "total_time", "type": "quantitative", "scale": {"type": "linear"}, "title": "time (s)"},
  row="context_strategy:N",
  color="update_strategy:N",
).properties(
  width=800,
  height=120
)

alt.Chart(...)

### Time series for each test case step

In [7]:
import altair as alt
import pandas as pd

all_timeseries_df = pd.DataFrame()

for c_strat in context_strategies:
  for u_strat in update_strategies:
    # check if that test case exists
    test_case = f"{c_strat[0]}-{u_strat[0]}.csv"
    if test_case not in available_test_cases:
      continue

    df = pd.read_csv(f"{path}/times/{test_case}")
    df["context_strategy"] = c_strat[0]
    df["update_strategy"] = u_strat[0]
    all_timeseries_df = all_timeseries_df.append(df)
    all_timeseries_df.reset_index(inplace=True, drop=True)


alt.Chart(all_timeseries_df).transform_fold(
  ["chunk_time", "storage_time", "context_time", "outdated_time", "old_doi_time", "store_new_time", 
   "update_dois_time", "total_time"]
).mark_line().encode(
  x="chunk:N",
  y="value:Q",
  color="key:N",
  column="update_strategy:N",
  row="context_strategy"
)

alt.Chart(...)